In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm, anderson
import seaborn as sns; sns.set()
from math import sqrt

# Ejercicio 5
Utilizando el generador implementado en el ejercicio 1:
* Implementar un método para generar variables aleatorias con distribución normal con media 15 y desvío 3.
* Graficar la distribución que siguen los números pseudo aleatorios generados.
* Realizar, al menos, 2 tests de los explicados en la materia para verificar si los números generados siguen la distribución pedida (evalué los resultados para distintos tamaños de muestra).

Empezamos implementando el método para generar las variables aleatorias. Para obtener la normal usamos el método de aceptación-rechazo

In [ ]:
def f1(u): # exponencial lambda = 1
    return -1*np.log(u)
def f2(x): # probabilidad de aceptar
    return np.exp(-1/2 * (x-1)**2 )

def signoPositivo():
    return np.random.rand() <= 0.5

def funcionNormal(media, desvio, N):
    U1 = np.random.rand(N) # vector aleatorio para aplicarle la exponencial
    U2 = np.random.rand(N) # vector aleatorio para comparar y aceptar o rechazar

    X=[]
    X11=[]
    for i in range(N):
        X1 = f1(U1[i])
        if (U2[i] <= f2(X1)): # aceptacion o rechazo
            X11.append(X1)
            if signoPositivo():
                X.append(X1 * desvio + media)
            else:
                X.append(-1 * X1 * desvio + media)

    return X, X11

v_normal, v_exponencial = funcionNormal(15,3,100000)

In [ ]:
#ploteamos la media exponencial   
plt.hist(v_exponencial,'sturges',color='orange')
plt.title('Histograma de media exponencial')
plt.xlabel('X1')
plt.show()

#ploteamos la normal
plt.hist( v_normal,'sturges')
plt.title('Histograma de Normal')
plt.xlabel('X')
plt.show()

print ('Media: ', np.mean(v_normal))
print ('Varianza: ', np.var(v_normal))

Vemos de realizar los tests para ver si se sigue la distribución pedida.

En primer lugar aplicamos el test Kolmogorov Smirnov, con los siguientes parámetros:

*   Nivel de significación $\alpha$ = 0,05
*   $K_{aceptación}$ tabulado usando K = C /$\sqrt{n}\ $ para el nivel de significacion C = 0,88



In [ ]:
def Kaceptacion(N):
  return 0.88/(sqrt(N))


def kolmogorov_smirnov(muestra, Kacept):
    k_mayor = 0
    k_menor = 0
    i = 1
    n = len(muestra)
    for x in sorted(muestra):#se ordena la muestra
        valor_teorico = norm.cdf(x, loc=15, scale=3) #valor teorico obtenido de la funcion normal
        k_mayor = max(k_mayor, (i / n - valor_teorico))
        k_menor = max(k_menor, (valor_teorico - (i -1) / n))
        i += 1

    k_mayor = sqrt(n) * k_mayor
    k_menor = sqrt(n) * k_menor
    k = max(k_mayor, k_menor)
    print("K obtenido = {0}".format(k))
    print("K aceptación = {0}".format(Kacept))
    if  k <= Kacept:
        print ('Acepto H0')
    else:
        print ('Rechazo H0')



Probamos el test para distintas muestras.

In [ ]:
for muestras in {1000, 10000, 100000, 500000}:
    print("Muestras: {0}".format(muestras))
    v_normal , exp = funcionNormal(15,3,muestras)
    kolmogorov_smirnov(v_normal,Kaceptacion(len(v_normal)))
    print("------")

In [ ]:
def andersonTest(v_normal):
    #Ejecutamos el test de normalidad(por defecto compara contra Normal)
  resultado = anderson(v_normal)

  print("Estadístico: %.3f" % resultado.statistic)

  for i in range(len(resultado.critical_values)):
    nivel_significancia, umbral= resultado.significance_level[i]/100, resultado.critical_values[i] #distintos niveles de significacion
    print("Nivel significancia: %.3f -- Umbral: %.3f" % (nivel_significancia, umbral))
    if resultado.statistic< resultado.critical_values[i]:
      print("Datos normales(No se puede rechazar H0)")
    else:
      print("Datos No normales(Se rechaza H0)")

In [ ]:
for muestras in {1000,10000,100000}:
  print("Muestras: {0}".format(muestras))
  v_normal , exp = funcionNormal(15,3,muestras)
  andersonTest(v_normal)
  print("------")
  